<a href="https://colab.research.google.com/github/cindy12651269/ExchangeRates_to_SQLite_with_Timestamp.ipynb/blob/cindy12651269%2Fhw1/ExchangeRates_to_SQLite_with_Timestamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from datetime import datetime

# Define the URL to scrape
url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'

# Send a GET request to the URL
r = requests.get(url)

# Parse the HTML content using BeautifulSoup with html5lib parser
soup = BeautifulSoup(r.text, 'html5lib')

# Find all the currency names
currency = soup.find_all('div', class_="visible-phone print_hide")

# Find all the cash buy rates
cash_buy_rate = soup.find_all('td', class_="rate-content-cash text-right print_hide", attrs={'data-table': '本行現金買入'})

# Find all the cash sell rates
cash_sell_rate = soup.find_all('td', class_='rate-content-cash text-right print_hide', attrs={'data-table': '本行現金賣出'})

# Find all the sight buy rates
sight_buy_rate = soup.find_all('td', class_='rate-content-sight text-right print_hide', attrs={'data-table': '本行即期買入'})

# Find all the sight sell rates
sight_sell_rate = soup.find_all('td', class_='rate-content-sight text-right print_hide', attrs={'data-table': '本行即期賣出'})

# Initialize an empty list to store the data
data_list = []

# Get the current timestamp
timestamp = datetime.now().strftime('%y-%m-%d %H:%M:%S')

for i in range(len(currency)):
    # Strip any whitespace characters in currency_names, cash_buy, cash_sell, sight_buy, and sight_sell
    currency_name = currency[i].text.strip()
    cash_buy = cash_buy_rate[i].text.strip()
    cash_sell = cash_sell_rate[i].text.strip()
    sight_buy = sight_buy_rate[i].text.strip()
    sight_sell = sight_sell_rate[i].text.strip()

    # Append the extracted data to the list along with the timestamp
    data_list.append([currency_name, cash_buy, cash_sell, sight_buy, sight_sell, timestamp])

# Create a DataFrame from the list with the specified column names
df = pd.DataFrame(data_list, columns=['幣別','現金買入','現金賣出','即期買入','即期賣出','更新時間'])

# Connect to the SQLite database
conn = sqlite3.connect('exchange_rates.db')

# Create a cursor object
c = conn.cursor()

# Create the exchange_rates table if it doesn't exist
c.execute(
    '''
    CREATE TABLE IF NOT EXISTS exchange_rates(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
         幣別 TEXT,
         現金買入 TEXT,
         現金賣出 TEXT,
         即期買入 TEXT,
         即期賣出 TEXT,
         更新時間 TEXT
    )
    '''
)

# Insert the data into the exchange_rates table
df.to_sql('exchange_rate', conn, if_exists='append', index=False)
# Commit the transaction and close the connection
conn.commit()
conn.close()

print(df)


            幣別     現金買入     現金賣出    即期買入    即期賣出               更新時間
0     美金 (USD)    32.15    32.82  32.475  32.625  24-07-09 13:50:43
1     港幣 (HKD)    4.011    4.215   4.132   4.202  24-07-09 13:50:43
2     英鎊 (GBP)     40.5    42.62  41.395  42.025  24-07-09 13:50:43
3     澳幣 (AUD)    21.54    22.32  21.755    22.1  24-07-09 13:50:43
4   加拿大幣 (CAD)    23.38    24.29   23.71   24.04  24-07-09 13:50:43
5   新加坡幣 (SGD)    23.52    24.43   23.99   24.21  24-07-09 13:50:43
6   瑞士法郎 (CHF)    35.45    36.65   36.06   36.45  24-07-09 13:50:43
7     日圓 (JPY)    0.193   0.2058  0.1998  0.2048  24-07-09 13:50:43
8    南非幣 (ZAR)        -        -    1.75    1.84  24-07-09 13:50:43
9    瑞典幣 (SEK)      2.7     3.22    3.03    3.15  24-07-09 13:50:43
10    紐元 (NZD)    19.46    20.31   19.79   20.09  24-07-09 13:50:43
11    泰幣 (THB)   0.7657   0.9557  0.8761  0.9221  24-07-09 13:50:43
12  菲國比索 (PHP)   0.4909   0.6229       -       -  24-07-09 13:50:43
13   印尼幣 (IDR)  0.00168  0.00238       -       -